In [156]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
from math import ceil, floor, sqrt

In [157]:
rootDir = os.getcwd()
trainDir = os.path.join(rootDir, 'antrenare')
templatesDir = os.path.join(rootDir, 'imagini_auxiliare/templates/')
trainImages = os.listdir(trainDir)
trainImages = sorted([image for image in trainImages if '.jpg' in image])
trainImagesCopy = trainImages.copy()
trainAnnotations = os.listdir(trainDir)
trainAnnotations = sorted([file for file in trainAnnotations if '.txt' in file])
trainAnnotationsCopy = trainAnnotations.copy()

In [158]:
def showImage(title, image, grayscale=True):
    colorMap = None
    if grayscale == True:
        colorMap = 'gray'
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    plt.imshow(image, cmap=colorMap)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [196]:
nGames = 4

nCells = 14
origBoard = np.full((nCells, nCells), -1)
origBoard[6][6] = 1
origBoard[6][7] = 2
origBoard[7][6] = 3
origBoard[7][7] = 4
def getNewBoard():
    return origBoard.copy()
multiplier = np.ones((nCells, nCells))
rows = [i for i in range(14)]
cols = [chr(ord('A') + i) for i in range(14)]

for i in range(5):
    multiplier[i][i] = 2
    multiplier[i + 9][i + 9] = 2
    multiplier[nCells - i - 1][i] = 2
    multiplier[i][nCells - i - 1] = 2

multiplier[0][0] = 3
multiplier[0][6] = 3
multiplier[0][7] = 3
multiplier[0][13] = 3
multiplier[6][0] = 3
multiplier[6][13] = 3
multiplier[7][0] = 3
multiplier[7][13] = 3
multiplier[13][0] = 3
multiplier[13][6] = 3
multiplier[13][7] = 3
multiplier[13][13] = 3

constraints = {
    (2, 5): "/",
    (2, 10): "/",
    (3, 6): "-",
    (3, 9): "-",
    (4, 7): "+",
    (4, 8): "*",
    (5, 2): "/",
    (5, 7): "*",
    (5, 8): "+",
    (5, 13): "/",
    (6, 3): "-",
    (6, 12): "-",
    (7, 4): "*",
    (7, 5): "+",
    (7, 10): "*",
    (7, 11): "+",
    (8, 4): "+",
    (8, 5): "*",
    (8, 10): "+",
    (8, 11): "*",
    (9, 3): "-",
    (9, 12): "-",
    (10, 2): "/",
    (10, 7): "+",
    (10, 8): "*",
    (10, 13): "/",
    (11, 7): "*",
    (11, 8): "+",
    (12, 6): "-",
    (12, 9): "-",
    (13, 5): "/",
    (13, 10): "/"    
}
 
pieces = {
    0: 1, 
    
    1: 7, 2: 7, 3: 7, 4: 7, 5: 7, 6: 7, 7: 7, 8: 7, 9: 7, 10: 7, 
    
    11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1,
    
    20: 1, 21: 1, 24: 1, 25: 1, 27: 1, 28: 1,
    
    30: 1, 32: 1, 35: 1, 36: 1,
    
    40: 1, 42: 1, 45: 1, 48: 1, 49: 1,
    
    50: 1, 54: 1, 56: 1,
    
    60: 1, 63: 1, 64: 1,
    
    70: 1, 72: 1,
    
    80: 1, 81: 1,
    
    90: 1
}

In [160]:
# Iterez prin jocuri - i
# pentru fiecare joc, citesc i_turns.txt
# iterez prin miscarile jucatorului
# la fiecare miscare, gasesc tabla
#                       gasesc noua piesa (diferenta de la tabla anterioara la cea actuala)
#                       identific piesa
#                       verific ecuatiile pe care le indeplineste
#                       aplic constrangeri si bonusuri
# 
# Game -> Turn -> round

In [161]:
def getBoard(img):
    boardMaskLow = (100, 50, 50)
    boardMaskHigh = (140, 255, 255)

    hsvImage = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    boardMask = cv.inRange(hsvImage, boardMaskLow, boardMaskHigh)

    contours, _ = cv.findContours(boardMask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    largestContour = max(contours, key=cv.contourArea)
    x, y, w, h = cv.boundingRect(largestContour)

    cutoutBoard = img[y : y + h, x : x + w]

    return cutoutBoard


In [162]:
def resizeBoard(board, width, height):
    board = cv.resize(board, (width, height), interpolation=cv.INTER_LINEAR)
    return board

In [163]:
def matchPiece(piece):
    # showImage('Piece trying to match', piece)
    correlations = []
    maxLocs = []
    pieceCopy = piece.copy()
    pieceCopy = cv.cvtColor(pieceCopy, cv.COLOR_BGR2GRAY)
    for i in pieces.keys():
        templateImagePath = os.path.join(templatesDir, f'{i}.jpg')
        templateImage = cv.imread(templateImagePath)
        pad = 10
        templateImage = templateImage[pad:-pad, pad:-pad]
        templateImage = cv.cvtColor(templateImage, cv.COLOR_BGR2GRAY)
        # try:
        corr = cv.matchTemplate(pieceCopy, templateImage,  cv.TM_CCOEFF_NORMED)
        # except:
        #     print(f'Dimensions: {piece.shape}, expected {templateImage.shape}')
        #     showImage('', piece)
        _, _, _, maxLoc = cv.minMaxLoc(corr)
        corr=np.max(corr)
        correlations.append(corr)
        maxLocs.append(maxLoc)

    maxLoc = maxLocs[np.argmax(correlations)]
    topLeft = maxLoc
    bottomRigth = (topLeft[0] + 100, topLeft[1] + 100)
    # cv.rectangle(piece, topLeft, bottomRigth, (0, 255, 0), 10)
    # showImage("Matched template", piece)

    return list(pieces.keys())[np.argmax(correlations)], topLeft

In [164]:
def getCells(img):
    cellsMaskLow = (90, 50, 190)
    cellsMaskHigh = (100, 100, 255)

    placedPiecesLow = (15,0,150)
    placedPiecesHigh = (80,60,255)

    hsvImage = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    cellsMask = cv.inRange(hsvImage, cellsMaskLow, cellsMaskHigh)
    placedPiecesMask = cv.inRange(hsvImage, placedPiecesLow, placedPiecesHigh)

    cellsMask = cv.bitwise_or(cellsMask, placedPiecesMask)
    # showImage('cells mask', cellsMask)
    kernel = np.ones((17, 17), np.uint8)
    cellsMask = cv.dilate(cellsMask, kernel)
    # showImage('cells mask', cellsMask)

    contours, _ = cv.findContours(cellsMask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    largestContour = max(contours, key=cv.contourArea)

    x, y, w, h = cv.boundingRect(largestContour)
    cellsOnly = img[y : y + h, x : x + w]

    # showImage('cells only', cellsOnly)

    return x, y, w, h

In [165]:
boardTemplatePath = os.path.join(templatesDir, '01.jpg')
boardTemplate = cv.imread(boardTemplatePath)
boardTemplate = getBoard(boardTemplate)
boardTemplate = resizeBoard(boardTemplate, 1980, 1980)

In [166]:
def getCellIndex(cellW, cellH, identifiedX, identifiedY, nCells):
    # i = int(identifiedY // cellH) + 2
    # j = int(identifiedX // cellW) + 2
    threshold = 0.6

    i = identifiedY / cellH
    j = identifiedX / cellW

    if i - (identifiedY // cellH) >= threshold:
        i = ceil(identifiedY / cellH)
    else:
        i = identifiedY // cellH
    if j - (identifiedX // cellW) >= threshold:
        j = ceil(identifiedX / cellW)
    else:
        j = identifiedX // cellW

    i += 1
    j += 1

    i = min(max(i, 1), nCells)
    j = min(max(j, 1), nCells)

    return i, chr(ord('A') + j - 1)

In [122]:
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 4
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        # hueHist = cv.calcHist(currentBoardHSV, [0], None, [255], [0, 255])
        # plt.subplot(211)
        # plt.plot(hueHist)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        # hueThreshold = cv.adaptiveThreshold(hueDifference,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
        _, hueThreshold = cv.threshold(hueDifference, 50, 255, cv.THRESH_BINARY)
        # showImage('Hue Threshold', hueThreshold)

    # aici ma gandeam ca poate ar trebui sa fac match direct cu huw threshold, ar putea fi o idee
        # testIdentification = matchPiece

        # cannyEdges = cv.Canny(hueThreshold, 50, 150)
        # showImage('canny', cannyEdges)
        # showImage('Hue Difference', hueDifference)
        # showImage('Hue Threshold', hueThreshold)

        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        # valueHist = cv.calcHist(currentBoardHSV, [0], None, [255], [0, 255])
        # plt.subplot(212)
        # plt.plot(valueHist)
        # plt.show()
        # showImage('Value Difference', valueDifference)
        # valueThreshold = cv.adaptiveThreshold(valueDifference,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
        _, valueThreshold = cv.threshold(valueDifference, 50, 255, cv.THRESH_BINARY)
        # _, valueThreshold = cv.threshold(valueDifference, 45, 60, cv.THRESH_BINARY)
        # # Accuracy for game 1: 96.0%
        # # Accuracy for game 2: 90.0%
        # # Accuracy for game 3: 94.0%
        # # Accuracy for game 4: 98.0%
        # # showImage('Value Threshold', valueThreshold)

        totalThreshold = cv.bitwise_or(hueThreshold, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(totalThreshold, kernel)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        largest_contour = max(contours, key=cv.contourArea)

        contours = sorted(contours, key=cv.contourArea, reverse=True)

        minDiff = 1e6
        targetArea = 1e4
        for contour in contours:
            contArea = cv.contourArea(contour)
            x, y, w, h = cv.boundingRect(contour)
            if contArea > targetArea and contArea - targetArea < minDiff and h / w > 0.9 and w / h > 0.9:
                minDiff = abs(contArea - targetArea)
                largest_contour = contour

        resizedCopy = resizedBoard.copy()
        cv.drawContours(resizedCopy, [largest_contour], -1, (0, 255, 0), 3)

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = min(w, 100)
            h = min(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]
        # showImage("Cropped Board", croppedBoard)

        identifiedPiece = matchPiece(croppedBoard)
        y, x = y + identifiedPiece[1][0], x + identifiedPiece[1][1]

        for offset in [20, 30, 40, 50, 60, 70]:
            i, j = getCellIndex(
                cellW, cellH, x - cellsX + offset, y - cellsY + offset, nCells
            )

            if offset == 20:
                if i == int(cellRow):
                    correctRowIndex20 += 1
                if j == cellCol:
                    correctColIndex20 += 1
            elif offset == 30:
                if i == int(cellRow):
                    correctRowIndex30 += 1
                if j == cellCol:
                    correctColIndex30 += 1
            elif offset == 40:
                if i == int(cellRow):
                    correctRowIndex40 += 1
                if j == cellCol:
                    correctColIndex40 += 1
            elif offset == 50:
                if i == int(cellRow):
                    correctRowIndex50 += 1
                if j == cellCol:
                    correctColIndex50 += 1
            elif offset == 60:
                if i == int(cellRow):
                    correctRowIndex60 += 1
                if j == cellCol:
                    correctColIndex60 += 1
            elif offset == 70:
                if i == int(cellRow):
                    correctRowIndex70 += 1
                if j == cellCol:
                    correctColIndex70 += 1

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
            # print(f'Identidied as {identifiedPiece[0]} instead of {int(value)}')
            # showImage('Hue Difference', hueDifference)
            # showImage('Hue Threshold', hueThreshold)
            # showImage('Value Difference', valueDifference)
            # showImage('Value Threshold', valueThreshold)
            # showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    # print(f'Row Accuracy: {correctRowIndex / 50 * 100}, Col Accuracy: {correctColIndex / 50 * 100}')
    # print(f"20: Row Accuracy: {correctRowIndex20 / 50 * 100}%, Col Accuracy: {correctColIndex20 / 50 * 100}%")
    # print(f"30: Row Accuracy: {correctRowIndex30 / 50 * 100}%, Col Accuracy: {correctColIndex30 / 50 * 100}%")
    # print(f"40: Row Accuracy: {correctRowIndex40 / 50 * 100}%, Col Accuracy: {correctColIndex40 / 50 * 100}%")
    # print(f"50: Row Accuracy: {correctRowIndex50 / 50 * 100}%, Col Accuracy: {correctColIndex50 / 50 * 100}%")
    # print(f"60: Row Accuracy: {correctRowIndex60 / 50 * 100}%, Col Accuracy: {correctColIndex60 / 50 * 100}%")
    # print(f"70: Row Accuracy: {correctRowIndex70 / 50 * 100}%, Col Accuracy: {correctColIndex70 / 50 * 100}%")

    # break

KeyboardInterrupt: 

In [167]:
def getMaxEnergyConcentration(board):
    kernel = np.full((10, 10), 1/10)
    energyConcentration = cv.filter2D(board, -1, kernel)
    return energyConcentration

In [ ]:
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 220, 255, cv.THRESH_BINARY)
        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        # resizedCopy = resizedBoard.copy()
        # cv.drawContours(resizedCopy, [largest_contour], -1, (0, 255, 0), 3)

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = min(w, 100)
            h = min(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]
        # showImage("Cropped Board", croppedBoard)

        identifiedPiece = matchPiece(croppedBoard)
        y, x = y + identifiedPiece[1][0], x + identifiedPiece[1][1]

        for offset in [20, 30, 40, 50, 60, 70]:
            i, j = getCellIndex(
                cellW, cellH, x - cellsX + offset - 40, y - cellsY + offset - 20, nCells
            )

            if offset == 20:
                if i == int(cellRow):
                    correctRowIndex20 += 1
                if j == cellCol:
                    correctColIndex20 += 1
            elif offset == 30:
                if i == int(cellRow):
                    correctRowIndex30 += 1
                if j == cellCol:
                    correctColIndex30 += 1
            elif offset == 40:
                if i == int(cellRow):
                    correctRowIndex40 += 1
                if j == cellCol:
                    correctColIndex40 += 1
            elif offset == 50:
                if i == int(cellRow):
                    correctRowIndex50 += 1
                if j == cellCol:
                    correctColIndex50 += 1
            elif offset == 60:
                if i == int(cellRow):
                    correctRowIndex60 += 1
                if j == cellCol:
                    correctColIndex60 += 1
            elif offset == 70:
                if i == int(cellRow):
                    correctRowIndex70 += 1
                if j == cellCol:
                    correctColIndex70 += 1

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
          # showImage(f'Sat Difference', satDifference)
          # showImage(f'Value Difference', valueDifference)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    # # print(f'Row Accuracy: {correctRowIndex / 50 * 100}, Col Accuracy: {correctColIndex / 50 * 100}')
    # print(f"20: Row Accuracy: {correctRowIndex20 / 50 * 100}%, Col Accuracy: {correctColIndex20 / 50 * 100}%")
    # print(f"30: Row Accuracy: {correctRowIndex30 / 50 * 100}%, Col Accuracy: {correctColIndex30 / 50 * 100}%")
    # print(f"40: Row Accuracy: {correctRowIndex40 / 50 * 100}%, Col Accuracy: {correctColIndex40 / 50 * 100}%")
    # print(f"50: Row Accuracy: {correctRowIndex50 / 50 * 100}%, Col Accuracy: {correctColIndex50 / 50 * 100}%")
    # print(f"60: Row Accuracy: {correctRowIndex60 / 50 * 100}%, Col Accuracy: {correctColIndex60 / 50 * 100}%")
    # print(f"70: Row Accuracy: {correctRowIndex70 / 50 * 100}%, Col Accuracy: {correctColIndex70 / 50 * 100}%")

    # break

Accuracy for game 1: 100.0%
Accuracy for game 2: 96.0%
Accuracy for game 3: 98.0%
Accuracy for game 4: 98.0%


poate match pe thresh

In [ ]:
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 3672000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        # resizedCopy = resizedBoard.copy()
        # cv.drawContours(resizedCopy, [largest_contour], -1, (0, 255, 0), 3)

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = min(w, 100)
            h = min(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]
        # showImage("Cropped Board", croppedBoard)

        identifiedPiece = matchPiece(croppedBoard)

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
        #   showImage('Max sat thresh', maxSatThresholded)
        #   showImage('Hue Difference', hueDifference)
        #   showImage(f'Sat Difference', satDifference)
        #   showImage(f'Value Difference', valueDifference)
        #   showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    # # print(f'Row Accuracy: {correctRowIndex / 50 * 100}, Col Accuracy: {correctColIndex / 50 * 100}')
    # print(f"20: Row Accuracy: {correctRowIndex20 / 50 * 100}%, Col Accuracy: {correctColIndex20 / 50 * 100}%")
    # print(f"30: Row Accuracy: {correctRowIndex30 / 50 * 100}%, Col Accuracy: {correctColIndex30 / 50 * 100}%")
    # print(f"40: Row Accuracy: {correctRowIndex40 / 50 * 100}%, Col Accuracy: {correctColIndex40 / 50 * 100}%")
    # print(f"50: Row Accuracy: {correctRowIndex50 / 50 * 100}%, Col Accuracy: {correctColIndex50 / 50 * 100}%")
    # print(f"60: Row Accuracy: {correctRowIndex60 / 50 * 100}%, Col Accuracy: {correctColIndex60 / 50 * 100}%")
    # print(f"70: Row Accuracy: {correctRowIndex70 / 50 * 100}%, Col Accuracy: {correctColIndex70 / 50 * 100}%")

    # break

Accuracy for game 1: 100.0%
Accuracy for game 2: 98.0%
Accuracy for game 3: 94.0%
Accuracy for game 4: 96.0%


In [ ]:
# 100x100 filtrul de max energy
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 2550000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = max(w, 100)
            h = max(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]

        identifiedPiece = matchPiece(croppedBoard)

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
        #   showImage('Max sat thresh', maxSatThresholded)
        #   showImage('Hue Difference', hueDifference)
        #   showImage(f'Sat Difference', satDifference)
        #   showImage(f'Value Difference', valueDifference)
        #   showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')

Accuracy for game 1: 100.0%
Accuracy for game 2: 98.0%
Accuracy for game 3: 98.0%
Accuracy for game 4: 98.0%


In [ ]:
# 100x100 filtrul de max energy
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 2550000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = max(w, 100)
            h = max(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]

        identifiedPiece = matchPiece(croppedBoard)

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
        #   showImage('Max sat thresh', maxSatThresholded)
        #   showImage('Hue Difference', hueDifference)
        #   showImage(f'Sat Difference', satDifference)
        #   showImage(f'Value Difference', valueDifference)
        #   showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')

Accuracy for game 1: 100.0%
Accuracy for game 2: 98.0%
Accuracy for game 3: 98.0%
Accuracy for game 4: 98.0%


In [168]:
def maximizeRightSideEnergy(board):
    kernel = np.full((5, 5), 0.5/5)
    kernel[:, 50:] = 1.5/5
    maximizedRightSideEnergy = cv.filter2D(board, -1, kernel)
    return maximizedRightSideEnergy

# def maximizeRightSideEnergy(board):
#     kernel = np.full((3, 3), 0.1/3)
#     kernel[:, 2] = 0.3/3
#     maximizedRightSideEnergy = cv.filter2D(board, -1, kernel)
#     return maximizedRightSideEnergy

In [ ]:
# 10x10 filtrul de max energy
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        # maxHueThresholded = maximizeRightSideEnergy(maxHueThresholded)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 2550000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = max(w, 100)
            h = max(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]

        identifiedPiece = matchPiece(croppedBoard)

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
        #   showImage('Max sat thresh', maxSatThresholded)
        #   showImage('Hue Difference', hueDifference)
        #   showImage(f'Sat Difference', satDifference)
        #   showImage(f'Value Difference', valueDifference)
        #   showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')

Accuracy for game 1: 100.0%
Accuracy for game 2: 100.0%
Accuracy for game 3: 98.0%
Accuracy for game 4: 100.0%


In [ ]:
# 10x10 filtrul de max energy
for game in range(1, nGames + 1):
    # game = 3
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)
        # showImage('Hue Thresh', hueThreshold)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        # showImage('satThreshold', satThreshold)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        # showImage('valueThreshold', valueThreshold)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        maxHueThresholded = maximizeRightSideEnergy(maxHueThresholded)
        kernel = np.ones((9, 9), np.uint8)
        maxHueThresholded = cv.erode(maxHueThresholded, kernel)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)
        # _, maxValueThresholded = cv.threshold(maxValueConcentration, 230, 255, cv.THRESH_BINARY)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 2550000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        # showImage('Max Hue Concentration', maxHueConentration)
        # showImage('Max Value Concentration', maxValueConcentration)
        # showImage('Max Combined Concentration', maxCombinedConcentration)
        # showImage('Max Dilated Concentration', maxDilatedConcentration)
        # showImage('Dilated Difference', dilatedDifference)
        # showImage('Max Hue Threshold', maxHueThresholded)
        # showImage('Max Value Threshold', maxValueThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            print(f'Game: {game}, Round: {roundNumber}')
            print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
            showImage('Previous board', previousRoundBoard)
            showImage('Current board', currentBoardHSV)
            showImage('hue th', hueThreshold)
            showImage('hue th', maxHueThresholded)
            showImage('sat thresh', maxSatThresholded)
            showImage('sat thresh', satThreshold)
            showImage('value thresh', valueThreshold)
            showImage('Dilated Difference', dilatedDifference)
            continue

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = max(w, 100)
            h = max(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]

        identifiedPiece = matchPiece(croppedBoard)

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        # else:
        #   showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
        #   print(f'Game: {game}, Round: {roundNumber}')
        #   print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
        #   # showImage(f'Combined Difference', combinedDiff)
        #   showImage(f'Dilated Difference', dilatedDifference)
        #   showImage('Max Hue Threshold', maxHueThresholded)
        #   showImage('Max sat thresh', maxSatThresholded)
        #   showImage('Hue Difference', hueDifference)
        #   showImage(f'Sat Difference', satDifference)
        #   showImage(f'Value Difference', valueDifference)
        #   showImage("Cropped Board", croppedBoard)

        previousRoundBoard = resizedBoard

        # iterations -= 1
        # if iterations == 0:
        #     break
    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    # break

Accuracy for game 1: 100.0%
Accuracy for game 2: 100.0%
Accuracy for game 3: 100.0%
Accuracy for game 4: 100.0%


In [ ]:
for game in range(1, nGames + 1):
    correctIdentifications = 0
    correctRowIndex20, correctColIndex20 = 0, 0
    correctRowIndex30, correctColIndex30 = 0, 0
    correctRowIndex40, correctColIndex40 = 0, 0
    correctRowIndex50, correctColIndex50 = 0, 0
    correctRowIndex60, correctColIndex60 = 0, 0
    correctRowIndex70, correctColIndex70 = 0, 0
    roundNumber = 0
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    iterations = 3
    previousRoundBoard = boardTemplate
    for img, annotation in zip(trainImages, trainAnnotations):
        roundNumber += 1
        image = cv.imread(os.path.join(trainDir, img))
        position, value = None, None
        with open(os.path.join(trainDir, annotation), 'r') as f:
            position, value = f.read().split()

        cellRow = position[:-1]
        cellCol = position[-1]

        cutoutBoard = getBoard(image)
        resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
        cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
        cellW, cellH = cellsW // nCells, cellsH // nCells

        currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
        previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

        hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
        _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)

        satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
        _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
        valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
        _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
        combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
        combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

        kernel = np.ones((3, 3), np.uint8)
        dilatedDifference = cv.dilate(combinedDiff, kernel)

        maxHueConentration = getMaxEnergyConcentration(hueThreshold)
        _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
        maxHueThresholded = maximizeRightSideEnergy(maxHueThresholded)
        kernel = np.ones((9, 9), np.uint8)
        maxHueThresholded = cv.erode(maxHueThresholded, kernel)
        pad = 30
        maxHueThresholded[: cellsY - pad, :] = 0
        maxHueThresholded[cellsY + cellsH + pad : , :] = 0
        maxHueThresholded[:, : cellsX - pad] = 0
        maxHueThresholded[:, cellsX + cellsW + pad :] = 0
        
        maxSatConentration = getMaxEnergyConcentration(satThreshold)
        _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
        maxSatThresholded[: cellsY - pad, :] = 0
        maxSatThresholded[cellsY + cellsH + pad : , :] = 0
        maxSatThresholded[:, : cellsX - pad] = 0
        maxSatThresholded[:, cellsX + cellsW + pad :] = 0

        maxValueConcentration = getMaxEnergyConcentration(valueThreshold)

        maxHSVThresholded = None
        if sum(maxHueThresholded.flatten()) < 2550000:
            maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
        else:
            maxHSVThresholded = maxHueThresholded
        dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

        contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        try:
            largest_contour = max(contours, key=cv.contourArea)
        except:
            continue

        x, y, w, h = cv.boundingRect(largest_contour)
        if (w >= 100 or h >= 100) and (w < 100 or h < 100):
            w = max(w, 100)
            h = max(h, 100)
        croppedBoard = resizedBoard[y:y+h, x:x+w]

        identifiedPiece = matchPiece(croppedBoard)
        y, x = y + identifiedPiece[1][0], x + identifiedPiece[1][1]

        for offset in [20, 30, 40, 50, 60, 70]:
            i, j = getCellIndex(
                cellW, cellH, x - cellsX + offset - 40, y - cellsY + offset - 20, nCells
            )

            if offset == 20:
                if i == int(cellRow):
                    correctRowIndex20 += 1
                if j == cellCol:
                    correctColIndex20 += 1
            elif offset == 30:
                if i == int(cellRow):
                    correctRowIndex30 += 1
                if j == cellCol:
                    correctColIndex30 += 1
            elif offset == 40:
                if i == int(cellRow):
                    correctRowIndex40 += 1
                if j == cellCol:
                    correctColIndex40 += 1
            elif offset == 50:
                if i == int(cellRow):
                    correctRowIndex50 += 1
                if j == cellCol:
                    correctColIndex50 += 1
            elif offset == 60:
                if i == int(cellRow):
                    correctRowIndex60 += 1
                if j == cellCol:
                    correctColIndex60 += 1
            elif offset == 70:
                if i == int(cellRow):
                    correctRowIndex70 += 1
                if j == cellCol:
                    correctColIndex70 += 1

        if identifiedPiece[0] == int(value):
            correctIdentifications += 1
        previousRoundBoard = resizedBoard

    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    print(f"20: Row Accuracy: {correctRowIndex20 / 50 * 100}%, Col Accuracy: {correctColIndex20 / 50 * 100}%")
    print(f"30: Row Accuracy: {correctRowIndex30 / 50 * 100}%, Col Accuracy: {correctColIndex30 / 50 * 100}%")
    print(f"40: Row Accuracy: {correctRowIndex40 / 50 * 100}%, Col Accuracy: {correctColIndex40 / 50 * 100}%")
    print(f"50: Row Accuracy: {correctRowIndex50 / 50 * 100}%, Col Accuracy: {correctColIndex50 / 50 * 100}%")
    print(f"60: Row Accuracy: {correctRowIndex60 / 50 * 100}%, Col Accuracy: {correctColIndex60 / 50 * 100}%")
    print(f"70: Row Accuracy: {correctRowIndex70 / 50 * 100}%, Col Accuracy: {correctColIndex70 / 50 * 100}%")


Accuracy for game 1: 100.0%
20: Row Accuracy: 100.0%, Col Accuracy: 100.0%
30: Row Accuracy: 100.0%, Col Accuracy: 100.0%
40: Row Accuracy: 100.0%, Col Accuracy: 100.0%
50: Row Accuracy: 100.0%, Col Accuracy: 100.0%
60: Row Accuracy: 76.0%, Col Accuracy: 100.0%
70: Row Accuracy: 24.0%, Col Accuracy: 100.0%
Accuracy for game 2: 100.0%
20: Row Accuracy: 100.0%, Col Accuracy: 100.0%
30: Row Accuracy: 100.0%, Col Accuracy: 100.0%
40: Row Accuracy: 100.0%, Col Accuracy: 100.0%
50: Row Accuracy: 100.0%, Col Accuracy: 100.0%
60: Row Accuracy: 72.0%, Col Accuracy: 100.0%
70: Row Accuracy: 18.0%, Col Accuracy: 100.0%
Accuracy for game 3: 100.0%
20: Row Accuracy: 100.0%, Col Accuracy: 100.0%
30: Row Accuracy: 100.0%, Col Accuracy: 100.0%
40: Row Accuracy: 100.0%, Col Accuracy: 100.0%
50: Row Accuracy: 100.0%, Col Accuracy: 100.0%
60: Row Accuracy: 90.0%, Col Accuracy: 100.0%
70: Row Accuracy: 34.0%, Col Accuracy: 100.0%
Accuracy for game 4: 100.0%
20: Row Accuracy: 100.0%, Col Accuracy: 100.0%
3

In [146]:
for game in range(1, nGames + 1):
    board = getNewBoard()
    correctIdentifications = 0
    turnsFile = os.path.join(trainDir, f'{game}_turns.txt')
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    previousRoundBoard = boardTemplate
    with open(turnsFile, 'r') as f:
        turnsContent = f.readlines()
    turnsContent.append('')
    line = turnsContent[0]
    player, turns = line.split()
    turns = int(turns)
    turnNumber = len(turnsContent)
    for i in range(1, turnNumber):
        turn = turnsContent[i]
        if turn != '':
            nextPlayer, nextTurns = turn.split()
            nextTurns = int(nextTurns)
        else:
            nextPlayer = 'final'
            nextTurns = 51
        identifiedPieces = []
        for img, annotation in zip(trainImages[turns - 1 : nextTurns - 1], trainAnnotations[turns - 1 : nextTurns - 1]):
            image = cv.imread(os.path.join(trainDir, img))
            position, value = None, None
            with open(os.path.join(trainDir, annotation), 'r') as f:
                position, value = f.read().split()

            cellRow = position[:-1]
            cellCol = position[-1]

            cutoutBoard = getBoard(image)
            resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
            cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
            cellW, cellH = cellsW // nCells, cellsH // nCells

            currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
            previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

            hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
            _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)

            satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
            _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
            valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
            _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
            combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
            combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

            kernel = np.ones((3, 3), np.uint8)
            dilatedDifference = cv.dilate(combinedDiff, kernel)

            maxHueConentration = getMaxEnergyConcentration(hueThreshold)
            _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
            maxHueThresholded = maximizeRightSideEnergy(maxHueThresholded)
            kernel = np.ones((9, 9), np.uint8)
            maxHueThresholded = cv.erode(maxHueThresholded, kernel)
            pad = 30
            maxHueThresholded[: cellsY - pad, :] = 0
            maxHueThresholded[cellsY + cellsH + pad : , :] = 0
            maxHueThresholded[:, : cellsX - pad] = 0
            maxHueThresholded[:, cellsX + cellsW + pad :] = 0
            
            maxSatConentration = getMaxEnergyConcentration(satThreshold)
            _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
            maxSatThresholded[: cellsY - pad, :] = 0
            maxSatThresholded[cellsY + cellsH + pad : , :] = 0
            maxSatThresholded[:, : cellsX - pad] = 0
            maxSatThresholded[:, cellsX + cellsW + pad :] = 0

            maxValueConcentration = getMaxEnergyConcentration(valueThreshold)

            maxHSVThresholded = None
            if sum(maxHueThresholded.flatten()) < 2550000:
                maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
            else:
                maxHSVThresholded = maxHueThresholded
            dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

            contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            try:
                largest_contour = max(contours, key=cv.contourArea)
            except:
                continue

            x, y, w, h = cv.boundingRect(largest_contour)
            if (w >= 100 or h >= 100) and (w < 100 or h < 100):
                w = max(w, 100)
                h = max(h, 100)
            croppedBoard = resizedBoard[y:y+h, x:x+w]

            identifiedPiece = matchPiece(croppedBoard)
            y, x = y + identifiedPiece[1][0], x + identifiedPiece[1][1]

            offset = 30
            i, j = getCellIndex(
                cellW, cellH, x - cellsX + offset - 40, y - cellsY + offset - 20, nCells
            )

            board[i - 1][ord(j) - ord('A') - 1] = identifiedPiece[0]
            pieces[identifiedPiece[0]] -= 1

            identifiedPieces.append(identifiedPiece[0])

            if identifiedPiece[0] == int(value):
                correctIdentifications += 1
            else:
                showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
                print(f'Game: {game}, Round: {roundNumber}')
                print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
                # showImage(f'Combined Difference', combinedDiff)
                showImage(f'Dilated Difference', dilatedDifference)
                showImage('Max Hue Threshold', maxHueThresholded)
                showImage('Max sat thresh', maxSatThresholded)
                showImage('Hue Difference', hueDifference)
                showImage(f'Sat Difference', satDifference)
                showImage(f'Value Difference', valueDifference)
                showImage("Cropped Board", croppedBoard)
            previousRoundBoard = resizedBoard

        print(f'Went from round number {turns} to {nextTurns}')
        print(f'Player {player}:')
        print(f'\tPlaced pieces: {identifiedPieces}')

        player, turns = nextPlayer, nextTurns

    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    print(board)

    break

Went from round number 1 to 5
Player Player1:
	Placed pieces: [8, 32, 40, 72]
Went from round number 5 to 11
Player Player2:
	Placed pieces: [8, 10, 80, 8, 10, 18]
Went from round number 11 to 14
Player Player1:
	Placed pieces: [3, 11, 14]
Went from round number 14 to 18
Player Player2:
	Placed pieces: [5, 3, 15, 12]
Went from round number 18 to 21
Player Player1:
	Placed pieces: [3, 7, 6]
Went from round number 21 to 24
Player Player2:
	Placed pieces: [9, 9, 3]
Went from round number 24 to 28
Player Player1:
	Placed pieces: [3, 10, 17, 49]
Went from round number 28 to 31
Player Player2:
	Placed pieces: [9, 63, 54]
Went from round number 31 to 34
Player Player1:
	Placed pieces: [36, 48, 27]
Went from round number 34 to 36
Player Player2:
	Placed pieces: [6, 7]
Went from round number 36 to 39
Player Player1:
	Placed pieces: [3, 2, 5]
Went from round number 39 to 43
Player Player2:
	Placed pieces: [7, 10, 70, 60]
Went from round number 43 to 48
Player Player1:
	Placed pieces: [2, 5, 10, 

In [203]:
def checkCombinations(position, constraint, currentBoard):
    i, j = position
    operations = []
    if constraint is not None:
        operations= [constraint]
    else:
        operations = ["+", "-", "*", "/"]

    print(f'received check for operations {operations}')
    
    possibleCombinations = []
    if i - 2 >= 0 and currentBoard[i - 2][j] != -1 and currentBoard[i - 1][j] != -1:
        possibleCombinations.append((currentBoard[i - 2][j], currentBoard[i - 1][j]))
    if j - 2 >= 0 and currentBoard[i][j - 2] != -1 and currentBoard[i][j - 1] != -1:
        possibleCombinations.append((currentBoard[i][j - 2], currentBoard[i][j - 1]))
    if i + 2 < nCells and currentBoard[i + 2][j] != -1 and currentBoard[i + 1][j] != -1:
        possibleCombinations.append((currentBoard[i + 1][j], currentBoard[i + 2][j]))
    if j + 2 < nCells and currentBoard[i][j + 2] != -1 and currentBoard[i][j + 1] != -1:
        possibleCombinations.append((currentBoard[i][j + 1], currentBoard[i][j + 2]))

    print(f'Checking combianations for {possibleCombinations}')

    satisfied = 0
    for combination in possibleCombinations:
        a, b = combination
        for operation in operations:
            result = None
            if operation == "+":
                result = a + b
            elif operation == "-":
                result = abs(a - b)
            elif operation == "/":
                try:
                    result = a / b if a > b else b / a
                except:
                    continue
            elif operation == "*":
                result = a * b

            if result == currentBoard[i][j]:
                print(f'satisfied {a} {operation} {b} == {currentBoard[i][j]}')
                satisfied += 1
                break

    return satisfied

In [204]:
def getScore(piece, position, currentBoard):
    i, j = position
    constraint = None
    if (i + 1, j + 1) in constraints:
        constraint = constraints[(i + 1, j + 1)]

    print(f'Checking combiations for {position, constraint}, with multiplier of {multiplier[i][j]}')

    return piece * multiplier[i][j] * checkCombinations(position, constraint, currentBoard)

In [205]:
for game in range(1, nGames + 1):
    board = getNewBoard()
    print(board)
    correctIdentifications = 0
    turnsFile = os.path.join(trainDir, f'{game}_turns.txt')
    trainImages = [image for image in trainImagesCopy if image.startswith(str(game))]
    trainAnnotations = [file for file in trainAnnotationsCopy if file.startswith(str(game))]
    previousRoundBoard = boardTemplate
    with open(turnsFile, 'r') as f:
        turnsContent = f.readlines()
    turnsContent.append('')
    line = turnsContent[0]
    player, turns = line.split()
    turns = int(turns)
    turnNumber = len(turnsContent)
    for i in range(1, turnNumber):
        score = 0
        turn = turnsContent[i]
        if turn != '':
            nextPlayer, nextTurns = turn.split()
            nextTurns = int(nextTurns)
        else:
            nextPlayer = 'final'
            nextTurns = 51
        identifiedPieces = []
        for img, annotation in zip(trainImages[turns - 1 : nextTurns - 1], trainAnnotations[turns - 1 : nextTurns - 1]):
            image = cv.imread(os.path.join(trainDir, img))
            position, value = None, None
            with open(os.path.join(trainDir, annotation), 'r') as f:
                position, value = f.read().split()

            cellRow = position[:-1]
            cellCol = position[-1]

            cutoutBoard = getBoard(image)
            resizedBoard = resizeBoard(cutoutBoard, 1980, 1980)
            cellsX, cellsY, cellsW, cellsH = getCells(resizedBoard)
            cellW, cellH = cellsW // nCells, cellsH // nCells

            currentBoardHSV = cv.cvtColor(resizedBoard, cv.COLOR_BGR2HSV)
            previousBoardHSV = cv.cvtColor(previousRoundBoard, cv.COLOR_BGR2HSV)

            hueDifference = cv.absdiff(currentBoardHSV[:, :, 0], previousBoardHSV[:, :, 0])
            _, hueThreshold = cv.threshold(hueDifference, 50, 60, cv.THRESH_BINARY)

            satDifference = cv.absdiff(currentBoardHSV[:, :, 1], previousBoardHSV[:, :, 1])
            _, satThreshold = cv.threshold(satDifference, 50, 55, cv.THRESH_BINARY)
            valueDifference = cv.absdiff(currentBoardHSV[:, :, 2], previousBoardHSV[:, :, 2])
            _, valueThreshold = cv.threshold(valueDifference, 50, 53, cv.THRESH_BINARY)
            combinedDiff = cv.bitwise_or(hueThreshold, satThreshold)
            combinedDiff = cv.bitwise_or(combinedDiff, valueThreshold)

            kernel = np.ones((3, 3), np.uint8)
            dilatedDifference = cv.dilate(combinedDiff, kernel)

            maxHueConentration = getMaxEnergyConcentration(hueThreshold)
            _, maxHueThresholded = cv.threshold(maxHueConentration, 220, 255, cv.THRESH_BINARY)
            maxHueThresholded = maximizeRightSideEnergy(maxHueThresholded)
            kernel = np.ones((9, 9), np.uint8)
            maxHueThresholded = cv.erode(maxHueThresholded, kernel)
            pad = 30
            maxHueThresholded[: cellsY - pad, :] = 0
            maxHueThresholded[cellsY + cellsH + pad : , :] = 0
            maxHueThresholded[:, : cellsX - pad] = 0
            maxHueThresholded[:, cellsX + cellsW + pad :] = 0
            
            maxSatConentration = getMaxEnergyConcentration(satThreshold)
            _, maxSatThresholded = cv.threshold(maxSatConentration, 240, 255, cv.THRESH_BINARY)
            maxSatThresholded[: cellsY - pad, :] = 0
            maxSatThresholded[cellsY + cellsH + pad : , :] = 0
            maxSatThresholded[:, : cellsX - pad] = 0
            maxSatThresholded[:, cellsX + cellsW + pad :] = 0

            maxValueConcentration = getMaxEnergyConcentration(valueThreshold)

            maxHSVThresholded = None
            if sum(maxHueThresholded.flatten()) < 2550000:
                maxHSVThresholded = cv.bitwise_or(maxHueThresholded, maxSatThresholded)
            else:
                maxHSVThresholded = maxHueThresholded
            dilatedDifference = cv.bitwise_and(dilatedDifference, maxHSVThresholded)

            contours, _ = cv.findContours(dilatedDifference,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

            try:
                largest_contour = max(contours, key=cv.contourArea)
            except:
                continue

            x, y, w, h = cv.boundingRect(largest_contour)
            if (w >= 100 or h >= 100) and (w < 100 or h < 100):
                w = max(w, 100)
                h = max(h, 100)
            croppedBoard = resizedBoard[y:y+h, x:x+w]

            identifiedPiece = matchPiece(croppedBoard)
            y, x = y + identifiedPiece[1][0], x + identifiedPiece[1][1]

            offset = 30
            i, j = getCellIndex(
                cellW, cellH, x - cellsX + offset - 40, y - cellsY + offset - 20, nCells
            )

            board[i - 1][ord(j) - ord('A')] = identifiedPiece[0]
            pieces[identifiedPiece[0]] -= 1

            identifiedPieces.append(identifiedPiece[0])
            print(f'\nGetting score for {identifiedPiece[0], (i - 1, ord(j) - ord('A'))}')
            score += getScore(identifiedPiece[0], (i - 1, ord(j) - ord('A')), board)

            if identifiedPiece[0] == int(value):
                correctIdentifications += 1
            else:
                showImage(f'Identified as {identifiedPiece[0]}', croppedBoard)
                print(f'Game: {game}, Round: {roundNumber}')
                print(f'Sum of thresholded pixels: {sum(maxHueThresholded.flatten())}')
                # showImage(f'Combined Difference', combinedDiff)
                showImage(f'Dilated Difference', dilatedDifference)
                showImage('Max Hue Threshold', maxHueThresholded)
                showImage('Max sat thresh', maxSatThresholded)
                showImage('Hue Difference', hueDifference)
                showImage(f'Sat Difference', satDifference)
                showImage(f'Value Difference', valueDifference)
                showImage("Cropped Board", croppedBoard)
            previousRoundBoard = resizedBoard

        print(f'Went from round number {turns} to {nextTurns}')
        print(f'Player {player}:')
        print(f'\tPlaced pieces: {identifiedPieces}')
        print(f'\n\tGot a score of {score}\n')

        player, turns = nextPlayer, nextTurns


    print(f'Accuracy for game {game}: {correctIdentifications / 50 * 100}%')
    print(board)

    break

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  1  2 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  3  4 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]

Getting score for (8, (8, 7))
Checking combiations for ((8, 7), None), with multiplier of 1.0
received check for operations ['+', '-', '*', '/']
Checking combianations for [(2, 4)]
satisfied 2 * 4 == 8

Getting score for (32, (9, 7))
Checking combiations for ((9, 7), '*'), with multiplier of 1.0
received check for operations ['*']
Checking combianations for [(4, 8)]

In [202]:
multiplier

array([[3., 1., 1., 1., 1., 1., 3., 3., 1., 1., 1., 1., 1., 3.],
       [1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.],
       [1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.],
       [1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
       [1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3.],
       [3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1.],
       [1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
       [1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.],
       [1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.],
       [3., 1., 1., 1., 1., 1., 3., 3., 1., 1., 1., 1., 1., 3.]])